In [2]:
import tweepy
import cnfg
from pymongo import MongoClient

In [5]:
config = cnfg.load(".twitter_config")
CONSUMER_KEY      = config["consumer_key"]
CONSUMER_SECRET   = config["consumer_secret"] 
OAUTH_TOKEN       = config["access_token"] 
OATH_TOKEN_SECRET = config["access_token_secret"] 

In [6]:
auth = tweepy.AppAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)

api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

if (not api):
    print("Can't Authenticate")
    sys.exit(-1)

In [7]:
client = MongoClient()
db = client.tweet_db

tweet_collection = db.tweet_collection_full

In [ ]:
import sys
import json
import os

# airlines = ["@united, '@Delta', '@SouthwestAir', '@AmericanAir']

searchQuery = '@AmericanAir'  # this is what we're searching for
maxTweets = 10000000 # Some arbitrary large number
tweetsPerQry = 100  # this is the max the API permits
fName = 'tweets.txt' # We'll store the tweets in a text file.

# If results from a specific ID onwards are reqd, set since_id to that ID.
# else default to no lower limit, go as far back as API allows
sinceId = '5a012783291ebbf619cd9b7a' # this is the last ID that from my previous search query


# If results only below a specific ID are, set max_id to that ID.
# else default to no upper limit, start from the most recent tweet matching the search query.
max_id = -1

tweetCount = 0
print("Downloading max {0} tweets".format(maxTweets))
while tweetCount < maxTweets:
    try:
        if (max_id <= 0):
            if (not sinceId):
                new_tweets = api.search(q=searchQuery, 
                                        count=tweetsPerQry, 
                                        tweet_mode='extended') # extended saves the full tweet instead of a truncated tweet
            else:
                new_tweets = api.search(q=searchQuery, 
                                        count=tweetsPerQry,
                                        since_id=sinceId, 
                                        tweet_mode='extended')
        else:
            if (not sinceId):
                new_tweets = api.search(q=searchQuery, 
                                        count=tweetsPerQry,
                                        max_id=str(max_id - 1), 
                                        tweet_mode='extended')
            else:
                new_tweets = api.search(q=searchQuery, 
                                        count=tweetsPerQry,
                                        max_id=str(max_id - 1),
                                        since_id=sinceId, 
                                        tweet_mode='extended')
        if not new_tweets:
            print("No more tweets found")
            break
            
        for tweet in new_tweets:
            try:
#                 tweet = json.loads(tweet)
                tweet_collection.insert(tweet._json) # insert tweets into mongoDB
                
            except BaseException as e:
                print('failed ondata,', str(e))
                time.sleep(5)
                pass
        tweetCount += len(new_tweets)
        print("Downloaded {0} tweets".format(tweetCount))
        max_id = new_tweets[-1].id
    except tweepy.TweepError as e:
        # Just exit if any error
        print("some error : " + str(e))
        break

print ("Downloaded {0} tweets".format(tweetCount))


In [8]:
api.rate_limit_status()['resources']['search']

{'/search/tweets': {'limit': 450, 'remaining': 450, 'reset': 1510504063}}

In [9]:
tweet_collection.find_one().keys()

dict_keys(['_id', 'created_at', 'id', 'id_str', 'full_text', 'truncated', 'display_text_range', 'entities', 'metadata', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'retweeted_status', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'lang'])